### XOR

In [1]:
import torch.nn.functional as F
from torch_explain.nn.concepts import ConceptReasoningLayer, IntpLinearLayer1, IntpLinearLayer2, IntpLinearLayer3
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch_explain as te
from torch_explain import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import logging
import warnings
warnings.filterwarnings('ignore')
# Set the logging level for matplotlib to WARNING to suppress debug information
logging.getLogger('matplotlib').setLevel(logging.WARNING)


PATH = 'models/'

2024-07-21 01:11:07,908 - matplotlib - DEBUG - matplotlib data path: d:\Github\pytorch_explain\env\Lib\site-packages\matplotlib\mpl-data
2024-07-21 01:11:07,915 - matplotlib - DEBUG - CONFIGDIR=C:\Users\a_h9\.matplotlib
2024-07-21 01:11:07,939 - matplotlib - DEBUG - interactive is False
2024-07-21 01:11:07,940 - matplotlib - DEBUG - platform is win32
2024-07-21 01:11:08,018 - matplotlib - DEBUG - CACHEDIR=C:\Users\a_h9\.matplotlib
2024-07-21 01:11:08,022 - matplotlib.font_manager - DEBUG - Using fontManager instance from C:\Users\a_h9\.matplotlib\fontlist-v390.json
2024-07-21 01:11:08,344 - matplotlib.pyplot - DEBUG - Loaded backend module://matplotlib_inline.backend_inline version unknown.
2024-07-21 01:11:08,348 - matplotlib.pyplot - DEBUG - Loaded backend inline version unknown.


In [2]:
from torch_explain.datasets import xor
x, c, y = xor(1000)

In [4]:
y = F.one_hot(y.long().ravel()).float()

DCR

In [7]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = ConceptReasoningLayer(embedding_size, y.shape[1])
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [8]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_DCRBase_XOR.pth')))

<All keys matched successfully>

In [9]:
cem = model[0]
dcr = model[1]
cem.eval()
dcr.eval()
c_emb , c_pred = cem(x)
y_pred, sign_attn, filter_attn = dcr(c_emb, c_pred, return_attn=True)


In [12]:
dcr.explain( c_emb, c_pred, mode='global')

[{'class': 'y_0', 'explanation': 'c_0 & c_1', 'count': 256},
 {'class': 'y_0', 'explanation': '~c_0 & ~c_1', 'count': 239},
 {'class': 'y_1', 'explanation': '~c_0 & c_1', 'count': 261},
 {'class': 'y_1', 'explanation': 'c_0 & ~c_1', 'count': 244}]

LLR 1

In [13]:
isBias = True

In [14]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer1(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [15]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR1_XOR.pth')))

<All keys matched successfully>

In [16]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [17]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([1000, 2]), torch.Size([1000, 2, 2]), torch.Size([1000, 2]))

In [22]:
dataframes = [pd.DataFrame() for _ in range(2)]
for i in range(2):
    dataframes[i]['c_0'] = c[:,0].detach().numpy()
    dataframes[i]['c_1'] = c[:,1].detach().numpy()
    dataframes[i]['w_0'] = weights[:,0,i].detach().numpy()
    dataframes[i]['w_1'] = weights[:,1,i].detach().numpy()
    dataframes[i]['bias'] = biases[:,i].detach().numpy()
    dataframes[i]['y'] = y[:,i].detach().numpy()

In [23]:
dataframes[0].head()

,c_0,c_1,w_0,w_1,bias,y
0,0.0,1.0,0.004096,0.000308,-10.592157,0.0
1,1.0,1.0,0.082607,0.965485,5.270632,1.0
2,0.0,0.0,0.941537,0.989170,8.935375,1.0
3,0.0,1.0,0.012790,0.000119,-11.170920,0.0
4,1.0,1.0,0.846210,0.226720,4.436595,1.0


LLR 2

In [24]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer2(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [26]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR2_XOR.pth')))

<All keys matched successfully>

In [27]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [28]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([1000, 2]), torch.Size([1000, 4, 2]), torch.Size([1000, 2]))

In [29]:
dataframes = [pd.DataFrame() for _ in range(2)]
for i in range(2):
    dataframes[i]['c_0'] = c[:,0].detach().numpy()
    dataframes[i]['c_1'] = c[:,1].detach().numpy()
    dataframes[i]['w_0'] = weights[:,0,i].detach().numpy()
    dataframes[i]['w_1'] = weights[:,1,i].detach().numpy()
    dataframes[i]['bias'] = biases[:,i].detach().numpy()
    dataframes[i]['y'] = y[:,i].detach().numpy()

In [30]:
dataframes[0].head()

,c_0,c_1,w_0,w_1,bias,y
0,0.0,1.0,0.002985,0.161829,-9.438059,0.0
1,1.0,1.0,0.864846,0.960241,5.419125,1.0
2,0.0,0.0,0.924686,0.907127,7.963484,1.0
3,0.0,1.0,0.017072,0.006331,-10.382553,0.0
4,1.0,1.0,0.946391,0.841722,5.438396,1.0


LLR 3

In [31]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer3(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [32]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR3_XOR.pth')))

<All keys matched successfully>

In [33]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [34]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([1000, 2]), torch.Size([1000, 2, 2]), torch.Size([1000, 2]))

In [35]:
dataframes = [pd.DataFrame() for _ in range(2)]
for i in range(2):
    dataframes[i]['c_0'] = c[:,0].detach().numpy()
    dataframes[i]['c_1'] = c[:,1].detach().numpy()
    dataframes[i]['w_0'] = weights[:,0,i].detach().numpy()
    dataframes[i]['w_1'] = weights[:,1,i].detach().numpy()
    dataframes[i]['bias'] = biases[:,i].detach().numpy()
    dataframes[i]['y'] = y[:,i].detach().numpy()

In [36]:
dataframes[0].head()

,c_0,c_1,w_0,w_1,bias,y
0,0.0,1.0,-0.935676,-0.867697,-7.471285,0.0
1,1.0,1.0,0.758933,0.958701,5.458310,1.0
2,0.0,0.0,0.860083,0.938507,10.202852,1.0
3,0.0,1.0,-0.802925,-0.995537,-10.512743,0.0
4,1.0,1.0,0.926309,0.840087,4.921591,1.0


_____